In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
path = 'data/enoe/2021/'
all_files = glob.glob(path + '/*.csv')

In [3]:
col_select = ['ent','ingocup','emp_ppal','clase2','clase3','sex','eda','eda7c','hrsocup',
              'scian','ambito2','tue2']

In [4]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, encoding = 'ISO-8859-1')
    df.columns = map(str.lower, df.columns)
    df = df[col_select]
   
    df = df[(df['clase2'] == 1) & (df['ingocup']>0)]
    df['ingocup_y'] = df['ingocup']*12
    
    df['ambito2'] = df['ambito2'].replace({2:'micronegocios',3:'micronegocios',
                                          4:'pequeños',5:'medianos',6:'grandes',
                                          7:'gobierno',8:'otros'})
    
    df['emp_ppal'] = df['emp_ppal'].replace({1:'informal',2:'formal'})
    
    df['tue2'] = df['tue2'].replace({1:'empresas en sociedad y corporativas',
                                    2:'negocios no costituidos en sociedad',
                                    3:'privadas', 4:'publicas', 5:'sector informal',
                                     6:'domestico remunerado', 7:'agricultura de subsistencia'
                                    })
    
    df['sex'] = df['sex'].replace({1:'hombre', 2:'mujer'})
    
    df["scian"] = df["scian"].replace({1: "agricultura, ganaderia, pesca, etc",2:"mineria",
                                                    3:"electricidad, agua y gas", 4:"construccion",
                                                    5:"industrias manufactureras", 6:"comercio al por mayor",
                                                    7:"comercio al por menor", 8:"transportes, correos y almacenamiento",
                                                    9:"informacion en medios masivos", 10:"servicios financieros y de seguros",
                                                    11:"servicios inmobiliarios y de alquiler",
                                                    12:"servicios profesionales cientificos y tecnicos",
                                                    13:"corporativos", 14:"servicios de apoyo a los negocios y manejo de desechos",
                                                    15:"servicios educativos",16:"servicios de salud y asistencia social",
                                                    17:"servicios de esparcimiento, culturales y deportivos",
                                                    18:"servicios de hospedaje y preparacion de alimentos y bebidas", 
                                                    19:"otros servicios, excepto actividades gubernamentales",
                                                    20:"actividades gubernamentales y de organismos internacionales",
                                                    21:"no especificado" })
    
    #df['anio'] = 2005
    
    li.append(df)

/tmp/ipykernel_17856/121383689.py:4: DtypeWarning: Columns (2,5,8,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0, encoding = 'ISO-8859-1')
/tmp/ipykernel_17856/121383689.py:4: DtypeWarning: Columns (2,5,8,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0, encoding = 'ISO-8859-1')
/tmp/ipykernel_17856/121383689.py:4: DtypeWarning: Columns (2,5,8,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0, encoding = 'ISO-8859-1')
/tmp/ipykernel_17856/121383689.py:4: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0, encoding = 'ISO-8859-1')


In [5]:
from functools import reduce

df_emp05 = reduce(lambda left, right: pd.concat(li),li)

In [6]:
df_emp05.shape

(488938, 13)

In [7]:
df_emp05['one'] = 1

## Pivot tables by labour subject

In [8]:
eu_size = pd.pivot_table(df_emp05, values =['one'], index=['ent'],
                        columns=['ambito2'], aggfunc = np.count_nonzero)

In [9]:
formal_ocup = pd.pivot_table(df_emp05, values = ['one'], index=['ent'],
                        columns=['emp_ppal'], aggfunc = np.count_nonzero)

In [10]:
income_by_for = pd.pivot_table(df_emp05, values = ['ingocup_y'], index=['ent'],
                        columns=['emp_ppal'], aggfunc = np.mean)

## Economic units by size

In [11]:
eu_size.head(1)

one                                                       
ambito2     0 gobierno grandes medianos micronegocios otros pequeños
ent                                                                 
1        2133      621    1970     1095          4009   515     1941

In [12]:
eu_size.columns = ['0','gobierno','grandes','medianos','micronegocios','otros','pequeños']

In [13]:
columns = eu_size.columns

In [14]:
eu_size['total'] = eu_size[columns].sum(axis=1)

In [15]:
eu_size_pct = eu_size[['0','gobierno','grandes','medianos','micronegocios','otros','pequeños']].div(eu_size.total,axis=0)

In [16]:
eu_size_pct = eu_size_pct.add_prefix('pct_')

In [17]:
eu_size_pct.head()

,pct_0,pct_gobierno,pct_grandes,pct_medianos,pct_micronegocios,pct_otros,pct_pequeños
ent,,,,,,,
1,0.173641,0.050554,0.160371,0.089140,0.326359,0.041924,0.158010
2,0.096170,0.024877,0.237459,0.103258,0.333424,0.040076,0.164736
3,0.115907,0.090909,0.074215,0.103884,0.375919,0.052504,0.186662
4,0.129656,0.076346,0.058032,0.088041,0.404457,0.056178,0.187290
5,0.038867,0.033200,0.245414,0.139098,0.331836,0.038993,0.172592


## Job's formality

In [18]:
formal_ocup.head(3)

one         
emp_ppal formal informal
ent                     
1          6980     5304
2          9230     5442
3          7160     4401

In [19]:
formal_ocup.columns = ['informal','formal']

In [20]:
columns = formal_ocup.columns

In [21]:
formal_ocup['total'] = formal_ocup[columns].sum(axis=1)

In [22]:
formal_pct = formal_ocup[['informal', 'formal']].div(formal_ocup.total,axis=0)

In [23]:
formal_pct.head(1)

,informal,formal
ent,,
1,0.568219,0.431781


In [24]:
formal_pct = formal_pct.add_prefix('pct_pob_')

## Income by formality condition

In [25]:
income_by_for.head(1)

ingocup_y              
emp_ppal         formal      informal
ent                                  
1         104464.590258  67087.653846

In [26]:
income_by_for.columns = ['informal','formal']

In [27]:
columns = income_by_for.columns

In [28]:
income_by_for.head()

,informal,formal
ent,,
1,104464.590258,67087.653846
2,129140.476706,89785.501654
3,148955.458659,94623.700068
4,131731.112296,63797.359075
5,118941.879737,74832.295665


In [29]:
income_by_for = income_by_for.add_prefix('ing_')

In [30]:
income_by_for.head(1)

,ing_informal,ing_formal
ent,,
1,104464.590258,67087.653846


## Concatenate

In [31]:
dataframe = pd.concat([income_by_for,eu_size_pct,formal_pct], axis=1)

In [32]:
dataframe.shape

(32, 11)

## Final details

In [33]:
dataframe.head()

,ing_informal,ing_formal,pct_0,pct_gobierno,pct_grandes,pct_medianos,pct_micronegocios,pct_otros,pct_pequeños,pct_pob_informal,pct_pob_formal
ent,,,,,,,,,,,
1,104464.590258,67087.653846,0.173641,0.050554,0.160371,0.089140,0.326359,0.041924,0.158010,0.568219,0.431781
2,129140.476706,89785.501654,0.096170,0.024877,0.237459,0.103258,0.333424,0.040076,0.164736,0.629089,0.370911
3,148955.458659,94623.700068,0.115907,0.090909,0.074215,0.103884,0.375919,0.052504,0.186662,0.619324,0.380676
4,131731.112296,63797.359075,0.129656,0.076346,0.058032,0.088041,0.404457,0.056178,0.187290,0.446823,0.553177
5,118941.879737,74832.295665,0.038867,0.033200,0.245414,0.139098,0.331836,0.038993,0.172592,0.645666,0.354334


In [34]:
dataframe['anio'] = 2021

In [35]:
dataframe['anio_text'] = dataframe['anio'].astype(str)

In [36]:
dataframe['ent_text'] = dataframe.index.astype(str)

In [37]:
dataframe['unique'] = dataframe['anio_text'] + '-' + dataframe['ent_text']

In [38]:
dataframe = dataframe.drop(['anio_text','ent_text'], axis=1)

In [39]:
dataframe.to_csv('2021_emp.csv')